In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
import sagemaker

In [2]:
session = boto3.Session(profile_name='default',region_name='us-east-2')

In [3]:
sagemaker_session = sagemaker.Session(boto_session=session)

In [4]:
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

In [5]:
bucket

'sagemaker-us-east-2-113516067754'

In [6]:
data_dir='../aws_test'

In [7]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix='test_data')

In [11]:
from sklearn.datasets import load_iris

In [12]:
iris=load_iris()

In [13]:
X=iris['data']
y=iris['target']

In [11]:
X.shape

(150, 4)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
lgr=LogisticRegression()

In [15]:
lgr.fit(X,y)

/Users/JoaoGomes/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [17]:
accuracy_score(lgr.predict(X),y)

0.9733333333333334

In [21]:
import pickle

In [22]:
pickle.dump(lgr,open('model_lgr.pkl','wb'))

In [48]:
input_data = sagemaker_session.upload_data(path='./model.tar.gz', bucket=bucket, key_prefix='model_lgr')

In [8]:
input_data='s3://sagemaker-us-east-2-113516067754/model_lgr/model.tar.gz'

# Deployment

In [6]:
from sagemaker.sklearn.model import SKLearnModel

In [21]:
import os, json

In [31]:
sklearn.__version__

'0.23.2'

In [7]:
role

'arn:aws:iam::113516067754:role/service-role/AmazonSageMaker-ExecutionRole-20200513T111076'

In [50]:
sklearn_model=SKLearnModel(model_data=input_data,
                             role=role,
                             entry_point="lgr_aws.py",
                             framework_version="0.23-1",
                            source_dir="aws_test",
                            dependencies=['./aws_test/model']
                            )

In [51]:
predictor=sklearn_model.deploy(initial_instance_count=1,instance_type='ml.t2.medium')

-----------------!

In [19]:
from sagemaker.predictor import json_serializer, json_deserializer

In [20]:
predictor.serializer=json_serializer
predictor.deserializer=json_deserializer

In [24]:
predictor.content_type='application/json'
predictor.accept='application/json'

In [29]:
data={'X':X[0:1].tolist()}

In [30]:
json.dumps(data)

'{"X": [[5.1, 3.5, 1.4, 0.2]]}'

In [25]:
pred=predictor.predict(data)

In [52]:
predictor.delete_endpoint()

In [54]:
load_iris.__module__

'sklearn.datasets._base'